<a href="https://colab.research.google.com/github/hukim1112/one-day-LLM/blob/main/6_Alpaca_finetunning_with_WandB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# From Llama to Alpaca: Finetunning and LLM with Weights & Biases

이 Notebook에서는 사전 훈련된 LLama 모델을 인스트럭션 데이터셋에 대해 미세 조정(fine-tuning)하는 방법을 배울 것입니다. davinci-003 (GPT-3)으로 생성된 데이터 대신 GPT-4를 사용하여 더욱 향상된 인스트럭션 데이터셋을 활용하는 업데이트된 버전의 Alpaca 데이터셋을 사용합니다. 자세한 내용은 [공식 저장소 페이지](https://github.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM#how-good-is-the-data)를 참조하세요.

이 Notebook은 최소 24GB 메모리를 갖춘 A100/A10 GPU가 필요합니다. 매개변수를 조정하여 T4에서 실행할 수도 있지만 실행 시간이 매우 길어집니다.

이 Notebook에는 연관 프로젝트인 알파카에 대한 보고서: [wandb](wandb.me/alpaca)가 있습니다.

In [ ]:
!pip install wandb
!pip install git+https://github.com/huggingface/transformers@v4.31-release
!pip install pip install accelerate -U

## Prepare your Instruction Dataset

인스트럭션 데이터셋은 사용자의 특정 도메인과 관련된 명령/결과 쌍의 목록입니다. 예를 들어, 특정 분야의 질문과 답변, 기술 분야의 문제와 해결책, 또는 단순히 명령과 결과가 될 수 있습니다. 일반적인 예로는 "jsonL 파일을 읽고 처음 5줄을 출력하는 파이썬 스크립트를 작성하라"가 있으며, 이때 모델은 다음과 유사한 내용을 출력할 수 있습니다:


```python
import json

fname = "my_file.json"

# read file from fname
with open(fname, "r") as f:
    data = json.load(f)

print(data[0:5])
```


the Alpaca (GPT-4 curated instructions and outputs) 데이터셋을 가져옵니다.:

In [ ]:
!wget https://raw.githubusercontent.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM/main/data/alpaca_gpt4_data.json

데이터셋을 로드합니다.

In [ ]:
import json

dataset_file = "alpaca_gpt4_data.json"

with open(dataset_file, "r") as f:
    alpaca = json.load(f)

In [ ]:
type(alpaca), alpaca[0:3]

In [ ]:
len(alpaca)

데이터셋에는 명령(instruction)과 결과(output)가 포함되어 있습니다. 모델은 다음 토큰을 예측하도록 훈련되므로, 한 가지 방법은 단순히 둘을 연결(concatenate)하고 그 결과를 토대로 모델을 훈련하는 것입니다. 이상적으로 프롬프트는 입력과 출력 위치를 명확하게 표시하는 방식으로 구성되어야 합니다. 모든 것을 체계적이게 유지하기 위해 dataset을 W&B (Weights & Biases)에 기록해 보겠습니다.

wandb는 "Weights & Biases"의 약자로, 머신 러닝 실험을 추적하고, 시각화하며, 공유하기 위한 툴입니다. 이 라이브러리는 머신러닝 프로젝트를 관리하기 위한 다양한 기능을 제공합니다.

### Train/Eval Split

In [ ]:
import random

seed = 42

random.seed(seed)
random.shuffle(alpaca)  # this could also be a parameter

In [ ]:
train_dataset = alpaca[:-1000]
eval_dataset = alpaca[-1000:]

데이터셋을 학습(training) 데이터셋과 평가(evaluation) 데이터셋으로 분할하고, 이를 JSON 형식으로 저장한 후, Weights & Biases(W&B) 플랫폼에 로깅합니다.

In [ ]:
import wandb

In [ ]:
import pandas as pd

train_df = pd.DataFrame(train_dataset)
eval_df = pd.DataFrame(eval_dataset)

train_table = wandb.Table(dataframe=train_df)
eval_table  = wandb.Table(dataframe=eval_df)

train_df.to_json("alpaca_gpt4_train.jsonl", orient='records', lines=True)
eval_df.to_json("alpaca_gpt4_eval.jsonl", orient='records', lines=True)

with wandb.init(project="alpaca_ft", job_type="split_data"):
    at = wandb.Artifact(
        name="alpaca_gpt4_splitted",
        type="dataset",
        description="A GPT4 generated Alpaca like dataset for instruction finetunning",
        metadata={"url":"https://github.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM#how-good-is-the-data"},
    )
    at.add_file("alpaca_gpt4_train.jsonl")
    at.add_file("alpaca_gpt4_eval.jsonl")
    wandb.log_artifact(at)
    wandb.log({"train_dataset":train_table, "eval_dataset":eval_table})

데이터셋을 테이블로 로깅하고 이것을 워크스페이스에 검사할 수 있습니다.

In [ ]:
def prompt_no_input(row):
    return ("Below is an instruction that describes a task. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Response:\n").format_map(row)

In [ ]:
row = alpaca[0]
print(prompt_no_input(row))

어떤 instruction은 input 변수 안에 context가 들어있습니다.

In [ ]:
row

In [ ]:
def prompt_input(row):
    return ("Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n").format_map(row)

In [ ]:
row = alpaca[9]
print(prompt_input(row))

일단은 프롬프트를 처리합니다. 나중에 적절한 양의 패딩(padding)과 함께 결과를 추가할 수 있습니다.

And the refactored function

In [ ]:
def create_alpaca_prompt(row):
    return prompt_no_input(row) if row["input"] == "" else prompt_input(row)

## Why are we doing all this?

우리가 업로드한 artifact에서 파일을 다시 로드할 수 있습니다.

In [ ]:
import json
from wandb import Api

api = Api()
artifact = api.artifact('capecape/alpaca_ft/alpaca_gpt4_splitted:v4', type='dataset')
dataset_dir = artifact.download()

def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data

train_dataset = load_jsonl(f"{dataset_dir}/alpaca_gpt4_train.jsonl")
eval_dataset = load_jsonl(f"{dataset_dir}/alpaca_gpt4_eval.jsonl")

아주 특정한 방법으로 토큰화를 해야만 모델이 결과(output)를 예측하도록 학습할 수 있기 때문입니다.

In [ ]:
train_prompts = [create_alpaca_prompt(row) for row in train_dataset]
eval_prompts = [create_alpaca_prompt(row) for row in eval_dataset]

In [ ]:
print(train_prompts[0])

우리는 target을 처리하고 문자열 종료 토큰(EOS)을 추가해야 합니다. LLama의 경우 이는: `"</s>"` 입니다.

In [ ]:
def pad_eos(ds):
    EOS_TOKEN = "</s>"
    return [f"{row['output']}{EOS_TOKEN}" for row in ds]

In [ ]:
train_outputs = pad_eos(train_dataset)
eval_outputs = pad_eos(eval_dataset)
train_outputs[0]

좋습니다! examples라는 변수에 최종 버전을 저장합시다.

In [ ]:
train_dataset = [{"prompt":s, "output":t, "example": s + t} for s, t in zip(train_prompts, train_outputs)]
eval_dataset = [{"prompt":s, "output":t, "example": s + t} for s, t in zip(eval_prompts, eval_outputs)]

이것이 모델이 보고 배울 필요가 있는 것입니다.

In [ ]:
print(train_dataset[0]["example"])

## Converting text to numbers: Tokenizer

우리는 데이터셋을 토큰들로 변환할 필요가 있습니다. 이것은 transformers 라이브러리로 쉽게 달성할 수 있습니다.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
#model_id = 'meta-llama/Llama-2-7b-hf
model_id = 'NousResearch/Llama-2-7b-chat-hf'
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
tokenizer.encode("My experiments are going strong!")

In [ ]:
tokenizer.encode("My experiments are going strong!", padding='max_length', max_length=10)

In [ ]:
tokenizer.encode("My experiments are going strong!",
                 padding='max_length',
                 max_length=10,
                 return_tensors="pt")

In [ ]:
tokenizer(["My experiments are going strong!",
           "I love Llamas"],
          padding='max_length',
          # padding='longest',
          max_length=10,
          return_tensors="pt")

### Packing

우리는 몇 개의 짧은 examples을 더 긴 chunk로 포장(packing)합니다. 이걸로 우리는 좀 더 효율적으로 학습할 수 있습니다.

여기서의 주요 아이디어는 지시사항/출력 샘플이 짧다는 것입니다. 그러니 EOS 토큰으로 구분지어 여러 개를 연결합시다. 데이터셋을 사전에 토큰화하고 사전에 패킹함으로써 모든 것을 더 빠르게 처리할 수 있습니다! 만약 max_seq_len = 1024로 설정한다면, 패킹하는 코드는 다음과 같이 보일 것입니다:

In [ ]:
max_sequence_len = 1024

def pack(dataset, max_seq_len=max_sequence_len):
    tkds_ids = tokenizer([s["example"] for s in dataset])["input_ids"]

    all_token_ids = []
    for tokenized_input in tkds_ids:
        all_token_ids.extend(tokenized_input)# + [tokenizer.eos_token_id])

    print(f"Total number of tokens: {len(all_token_ids)}")
    packed_ds = []
    for i in range(0, len(all_token_ids), max_seq_len+1):
        input_ids = all_token_ids[i : i + max_seq_len+1]
        if len(input_ids) == (max_seq_len+1):
            packed_ds.append({"input_ids": input_ids[:-1], "labels": input_ids[1:]})  # this shift is not needed if using the model.loss
    return packed_ds

In [ ]:
train_ds_packed = pack(train_dataset)
eval_ds_packed = pack(eval_dataset)
len(train_ds_packed)

In [ ]:
eval_ds_packed

이렇게 하면, 길이가 1024인 11,000개 이상의 시퀀스를 얻게 됩니다.



---



### DataLoader
일반적인 크로스 엔트로피로 훈련을 하고 이 패킹된 데이터셋에서 다음 토큰을 예측할 것입니다.

In [ ]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

torch.manual_seed(seed)
batch_size = 16  # I have an A100 GPU with 40GB of RAM 😎

train_dataloader = DataLoader(
    train_ds_packed,
    batch_size=batch_size,
    collate_fn=default_data_collator, # we don't need any special collator 😎
)

eval_dataloader = DataLoader(
    eval_ds_packed,
    batch_size=batch_size,
    collate_fn=default_data_collator,
    shuffle=False,
)

batch가 어떻게 생겼는지를 확인합니다. 데이터로더로부터 다음과 같이 샘플링할 수 있습니다.

In [ ]:
b = next(iter(train_dataloader))
b

We can alos decode the batch just to be super sure

In [ ]:
tokenizer.decode(b["input_ids"][0])[:250]

In [ ]:
tokenizer.decode(b["labels"][0])[:250]

## Train

다음과 같이 모든 하이퍼파라미터들을 관리합니다.

In [ ]:
from types import SimpleNamespace

gradient_accumulation_steps = 2

config = SimpleNamespace(
    model_id=model_id,
    dataset_name="alpaca-gpt4",
    precision="bf16",  # faster and better than fp16, requires new GPUs
    n_freeze=24,  # How many layers we don't train, LLama 7B has 32.
    lr=2e-4,
    n_eval_samples=10, # How many samples to generate on validation
    max_seq_len=max_sequence_len, # Lenght of the sequences to pack
    epochs=3,  # we do 3 pasess over the dataset.
    gradient_accumulation_steps=gradient_accumulation_steps,  # evey how many iterations we update the gradients, simulates larger batch sizes
    batch_size=batch_size,  # what my GPU can handle, depends on how many layers are we training
    log_model=False,  # upload the model to W&B?
    gradient_checkpointing = True,  # saves even more memory
    freeze_embed = True,  # why train this? let's keep them frozen ❄️
    seed=seed,
)

config.total_train_steps = config.epochs * len(train_dataloader) // config.gradient_accumulation_steps

In [ ]:
print(f"We will train for {config.total_train_steps} steps and evaluate every epoch")

pretrained model을 가져옵니다.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    config.model_id,
    device_map=0,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16,
    use_cache=False,
)

In [ ]:
def param_count(m):
    params = sum([p.numel() for p in m.parameters()])/1_000_000
    trainable_params = sum([p.numel() for p in m.parameters() if p.requires_grad])/1_000_000
    print(f"Total params: {params:.2f}M, Trainable: {trainable_params:.2f}M")
    return params, trainable_params

params, trainable_params = param_count(model)

전체 모델을 학습하는 것은 강력한 연산력과 메모리를 필요로하기 때문에 우리는 8개의 layer를 튜닝할 것 입니다. LLama는 총 32개를 가지고 있습니다.

In [ ]:
# freeze layers (disable gradients)
for param in model.parameters(): param.requires_grad = False
for param in model.lm_head.parameters(): param.requires_grad = True
for param in model.model.layers[config.n_freeze:].parameters(): param.requires_grad = True

In [ ]:
# Just freeze embeddings for small memory decrease
if config.freeze_embed:
    model.model.embed_tokens.weight.requires_grad_(False);

또한 그래디언트 체크포인팅을 사용하여 더 많이 저장할 수도 있습니다(이것은 훈련을 느리게 만들지만, 얼마나 느려질지는 여러분의 특정 설정에 따라 달라집니다). 대용량 모델을 메모리에 맞추는 방법에 대해 허깅페이스 웹사이트에 [좋은 아티클](https://huggingface.co/docs/transformers/v4.18.0/en/performance)이 있으니 확인해 보시길 권장합니다!


In [ ]:
# save more memory
if config.gradient_checkpointing:
    model.gradient_checkpointing_enable()

In [ ]:
params, trainable_params = param_count(model)

### Optimizer


In [ ]:
from transformers import get_cosine_schedule_with_warmup

optim = torch.optim.Adam(model.parameters(), lr=config.lr, betas=(0.9,0.99), eps=1e-5)
scheduler = get_cosine_schedule_with_warmup(
    optim,
    num_training_steps=config.total_train_steps,
    num_warmup_steps=config.total_train_steps // 10,
)

In [ ]:
def loss_fn(x, y):
    "A Flat CrossEntropy"
    return torch.nn.functional.cross_entropy(x.view(-1, x.shape[-1]), y.view(-1))

## Testing during training

거의 다 왔습니다, 이제 모델에서 샘플링하는 간단한 함수를 만들어 가끔 모델이 출력하는 것을 시각적으로 확인해 봅시다! 간단하게 모델.generate 메소드를 감싸 보겠습니다. GenerationConfig에서 기본 샘플링 매개변수를 가져와 해당 모델 ID를 전달하면 됩니다.


In [ ]:
from types import SimpleNamespace
from transformers import GenerationConfig

gen_config = GenerationConfig.from_pretrained(config.model_id)
test_config = SimpleNamespace(
    max_new_tokens=256,
    gen_config=gen_config)

In [ ]:
def generate(prompt, max_new_tokens=test_config.max_new_tokens, gen_config=gen_config):
    tokenized_prompt = tokenizer(prompt, return_tensors='pt')['input_ids'].cuda()
    with torch.inference_mode():
        output = model.generate(tokenized_prompt,
                            max_new_tokens=max_new_tokens,
                            generation_config=gen_config)
    return tokenizer.decode(output[0][len(tokenized_prompt[0]):], skip_special_tokens=True)

LoL 🤷

In [ ]:
prompt = eval_dataset[14]["prompt"]
print(prompt + generate(prompt, 128))

우리는 그 결과를 n 단계마다 프로젝트에 테이블로 기록할 수 있습니다.

In [ ]:
import wandb
from tqdm.auto import tqdm

def prompt_table(examples, log=False, table_name="predictions"):
    table = wandb.Table(columns=["prompt", "generation", "concat", "output", "max_new_tokens", "temperature", "top_p"])
    for example in tqdm(examples, leave=False):
        prompt, gpt4_output = example["prompt"], example["output"]
        out = generate(prompt, test_config.max_new_tokens, test_config.gen_config)
        table.add_data(prompt, out, prompt+out, gpt4_output, test_config.max_new_tokens, test_config.gen_config.temperature, test_config.gen_config.top_p)
    if log:
        wandb.log({table_name:table})
    return table

def to_gpu(tensor_dict):
    return {k: v.to('cuda') for k, v in tensor_dict.items()}

class Accuracy:
    "A simple Accuracy function compatible with HF models"
    def __init__(self):
        self.count = 0
        self.tp = 0.
    def update(self, logits, labels):
        logits, labels = logits.argmax(dim=-1).view(-1).cpu(), labels.view(-1).cpu()
        tp = (logits == labels).sum()
        self.count += len(logits)
        self.tp += tp
        return tp / len(logits)
    def compute(self):
        return self.tp / self.count

원하신다면 검증을 빠르게 추가할 수도 있습니다. 이 단계에서 테이블을 생성할 수도 있습니다.

In [ ]:
@torch.no_grad()
def validate():
    model.eval();
    eval_acc = Accuracy()
    loss, total_steps = 0., 0
    for step, batch in enumerate(pbar:=tqdm(eval_dataloader, leave=False)):
        pbar.set_description(f"doing validation")
        batch = to_gpu(batch)
        total_steps += 1
        with torch.amp.autocast("cuda", dtype=torch.bfloat16):
            out = model(**batch)
            loss += loss_fn(out.logits, batch["labels"])  # you could use out.loss and not shift the dataset
        eval_acc.update(out.logits, batch["labels"])
    # we log results at the end
    wandb.log({"eval/loss": loss.item() / total_steps,
               "eval/accuracy": eval_acc.compute()})
    prompt_table(eval_dataset[:config.n_eval_samples], log=True)
    model.train();

In [ ]:
from pathlib import Path
def save_model(model, model_name, models_folder="models", log=False):
    """Save the model to wandb as an artifact
    Args:
        model (nn.Module): Model to save.
        model_name (str): Name of the model.
        models_folder (str, optional): Folder to save the model. Defaults to "models".
    """
    model_name = f"{wandb.run.id}_{model_name}"
    file_name = Path(f"{models_folder}/{model_name}")
    file_name.parent.mkdir(parents=True, exist_ok=True)
    model.save_pretrained(file_name, safe_serialization=True)
    # save tokenizer for easy inference
    tokenizer = AutoTokenizer.from_pretrained(model.name_or_path)
    tokenizer.save_pretrained(model_name)
    if log:
        at = wandb.Artifact(model_name, type="model")
        at.add_dir(file_name)
        wandb.log_artifact(at)

모델 평가와 모델 출력을 table에 기록하는 루프를 정의합니다.

## The actual Loop
- 그래디언트 누적 및 그래디언트 스케일링
- 샘플링 및 모델 체크포인트 저장 (이것은 매우 빠르게 훈련되므로 여러 체크포인트를 저장할 필요가 없습니다)
- 우리는 토큰 정확도를 계산합니다, 손실보다 더 나은 지표입니다.

In [ ]:
wandb.init(project="alpaca_ft", # the project I am working on
           tags=["baseline","7b"],
           job_type="train",
           config=config) # the Hyperparameters I want to keep track of

# Training
acc = Accuracy()
model.train()
train_step = 0
for epoch in tqdm(range(config.epochs)):
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = to_gpu(batch)
        with torch.amp.autocast("cuda", dtype=torch.bfloat16):
            out = model(**batch)
            loss = loss_fn(out.logits, batch["labels"]) / config.gradient_accumulation_steps  # you could use out.loss and not shift the dataset
            loss.backward()
        if step%config.gradient_accumulation_steps == 0:
            # we can log the metrics to W&B
            wandb.log({"train/loss": loss.item() * config.gradient_accumulation_steps,
                       "train/accuracy": acc.update(out.logits, batch["labels"]),
                       "train/learning_rate": scheduler.get_last_lr()[0],
                       "train/global_step": train_step})
            optim.step()
            scheduler.step()
            optim.zero_grad(set_to_none=True)
            train_step += 1
    validate()

In [ ]:
# we save the model checkpoint at the end
#config.do_sample = True  # 샘플링을 활성화합니다.

# del config.temperature  # temperature 설정을 제거합니다.
# del config.top_p  # top_p 설정을 제거합니다.
save_model(model, model_name=config.model_id.replace("/", "_"), models_folder="models/", log=config.log_model)

wandb.finish()

A100에서 약 70분 정도 소요됩니다.

## Full Eval Dataset evaluation

평가 데이터셋(eval_dataset)에서 모델 예측을 로그하는 테이블을 만들어 보겠습니다 (처음 250개 샘플에 대해서).

In [ ]:
with wandb.init(project="alpaca_ft", # the project I am working on
           job_type="eval",
           config=config): # the Hyperparameters I want to keep track of
    model.eval();
    prompt_table(eval_dataset[:250], log=True, table_name="eval_predictions")